# Testing performance of Numba implementation of GreedySearch without the objective function as a higher order function

## Imports

In [1]:
from realkd.rules import RuleBoostingEstimator, XGBRuleEstimator, logistic_loss
from realkd.search import search_methods
from numba import njit, parallel_chunksize
import pandas as pd
import sortednp as snp
import doctest
from realkd.search import GreedySearch
import matplotlib.pyplot as plt

from collections import defaultdict, deque
from sortedcontainers import SortedSet
from math import inf
from heapq import heappop, heappush
from numpy import array
from bitarray import bitarray
import collections.abc
from numba.typed import List
from math import inf
from numpy import arange, argsort, array, cumsum, exp, full_like, log2, stack, zeros, zeros_like
from pandas import qcut, Series
import time
from sklearn.base import BaseEstimator, clone

from realkd.search import Conjunction, Context, KeyValueProposition, Constraint, NumbaGreedySearch
from bitarray.util import subset
import numpy as np
import pandas as pd

from realkd.logic import Conjunction, Constraint, KeyValueProposition, TabulatedProposition

## Test helpers

In [2]:
RNG = np.random.default_rng(seed=0)
ALPHA = 0.5

def rand_array(size, alpha=0.2):
    n, k = size
    d = np.arange(n*k)
    RNG.shuffle(d)
    d = (d < alpha*len(d)).astype(int)
    return d.reshape(n, k)

def generate_test_data(m, n):
    """
     :param int n: number of columns
     :param int m: number of rows
    """
    X = rand_array((m, n), alpha=ALPHA)
    true_weights = RNG.random(n) * 10
    y = X @ true_weights + RNG.random(m)
    y = np.sign(y - y.mean())
    dfX = pd.DataFrame(data=X, index=None, columns=[f'x{i}' for i in range(X.shape[1])])
    dfy = pd.Series(data=y)
    return (dfX, dfy)

In [3]:
def get_ghregn(X, y):
    """
    This would be part of the objective e.g GradientBoostingObjective
    """
    # The following is only ONE example of this class of objective function, it would be different for
    # different X, y, losses, regs and predictions.
    loss = logistic_loss
    reg = 1.0
    predictions = zeros_like(y)
    g = array(loss.g(y, predictions))
    h = array(loss.h(y, predictions))
    r = g / h
    order = argsort(r)[::-1]
    g = g[order]
    h = h[order]
    n = y.shape[0]

    def objective_function(ext):
        if len(ext) == 0:
            return -inf
        g_q = g[ext]
        h_q = h[ext]
        return g_q.sum() ** 2 / (2 * n * (reg + h_q.sum()))

    return g, h, reg, objective_function

In [4]:
def run_search_numba(ctx, g, h, reg, obj_fn):
    search = NumbaGreedySearch(ctx=ctx, bdn=None, g=g, h=h, reg=reg)
    search.run()

def run_search_base(ctx, g, h, reg, obj_fn):
    search = GreedySearch(ctx=ctx, obj=obj_fn, bdn=None)
    search.run()

In [5]:
class FakeContext:
    def extension(self, intent):
        if not intent:
            return array(range(self.m))

        raise Exception('Not implemented')
    
    def __init__(self, X):
        m, n = X.shape
        print(m, n, X.iat[2, 2])
        self.m = m
        self.n = n
        self.attributes = [KeyValueProposition(f'x{i}', Constraint.equals(1)) for i in range(1100)]
        self.extents = [np.array([i for i in range(self.m) if (X.iat[i, j] == 1)], dtype='int64') for j in range(self.n)]
        print(self.extents)

def get_full_context(X, y):
    ctx = FakeContext(X)
    g, h, reg, obj_fn = get_ghregn(X, y)
    print(g, h)
    return ctx, g, h, reg, obj_fn

## Testing config

In [6]:
import timeit


ns = np.arange(50, 501, 50)
ms = np.arange(500, 1101, 200)

ms_to_plot = np.arange(5000, 500001, 5000)
ns_to_plot = np.arange(500, 1101, 200)


LOOPS_PER_RUN=100
NUM_REPEATS=5
RUNS_PER_REPEAT=5

benchmark_configs = [
  {
    'name': 'benchmarks_n',
    'display': {
      'title': 'Varying n (#columns)',
      'xlabel': '$n$ (m = 5000)'
    },
    'iterator': ns_to_plot,
    'config': {
      'n': '$i',
      'm': 5000
    }
  },
  {
    'name': 'benchmarks_m',
    'display': {
      'title': 'Varying m (#rows)',
      'xlabel': '$m$ (n = 500)'
    },
    'iterator': ms_to_plot,
    'config': {
      'n': 500,
      'm': '$i'
    }
  },
]

benchmark_results = {}

# Run all functions first to eliminate the effect of compilation
def precompile_all():
  X, y = generate_test_data(100, 101)
  ctx, g, h, reg, obj_fn = get_full_context(X, y)
  run_search_numba(ctx, g, h, reg, obj_fn)
  run_search_base(ctx, g, h, reg, obj_fn)


precompile_all()

for benchmark in benchmark_configs:
  print(benchmark['name'])
  benchmark_results[benchmark['name']] = {
    'run_search_numba': {},
    'run_search_base': {}
  }
  for i in benchmark['iterator']:
    for method in ['run_search_numba', 'run_search_base']:
      print(f'i={i}, method={method}:')
      n = i if benchmark['config']['n'] == '$i' else benchmark['config']['n']
      m = i if benchmark['config']['m'] == '$i' else benchmark['config']['m']
      timeresults = []
      for repeat in range(NUM_REPEATS):
        print('generating test data')
        X, y = generate_test_data(m, n)
        print('getting context')
        ctx, g, h, reg, obj_fn = get_full_context(X, y)
        print('running test')
        timeresults.append(min(timeit.repeat(f'{method}(ctx, g, h, reg, obj_fn)', repeat=RUNS_PER_REPEAT, number=LOOPS_PER_RUN, globals=globals())))
      benchmark_results[benchmark['name']][method][i] = timeresults

100 101 0
[array([ 1,  4,  5,  7,  9, 10, 11, 13, 15, 18, 19, 20, 22, 23, 26, 28, 29,
       33, 34, 36, 39, 42, 43, 46, 48, 50, 51, 56, 59, 62, 64, 66, 68, 69,
       70, 71, 72, 74, 75, 77, 82, 84, 85, 86, 87, 90, 91, 93, 94, 96, 97,
       98, 99]), array([ 0,  1,  3,  6,  8,  9, 10, 11, 13, 16, 18, 19, 20, 21, 22, 23, 24,
       26, 29, 31, 32, 39, 41, 42, 44, 46, 47, 51, 55, 56, 59, 69, 73, 75,
       79, 80, 81, 82, 83, 84, 88, 89, 90, 95, 96, 97, 99]), array([ 3,  4, 12, 13, 14, 16, 27, 29, 30, 31, 32, 34, 35, 36, 39, 40, 43,
       44, 46, 47, 48, 50, 53, 57, 58, 61, 62, 65, 66, 67, 70, 73, 74, 75,
       77, 78, 81, 83, 85, 87, 89, 90, 92, 93, 95, 96]), array([ 0,  1,  2,  3,  5,  9, 11, 14, 15, 18, 19, 20, 21, 22, 23, 26, 28,
       30, 31, 32, 33, 34, 37, 39, 40, 41, 42, 45, 46, 53, 54, 57, 58, 60,
       61, 62, 65, 68, 73, 74, 76, 77, 78, 82, 83, 84, 86, 88, 90, 91, 93,
       95, 96, 97, 98, 99]), array([ 0,  1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 12, 19, 20, 24, 25, 26,


/mnt/c/Users/locke/Data/BE/realkd.py/realkd/search.py:722: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "run_greedy_search_gradient_boosting" failed type inference due to: Invalid use of type(CPUDispatcher(<function gradient_boosting_objective_function at 0x7eff6c42ba60>)) with parameters (int64, array(float64, 1d, C), array(float64, 1d, C), float64, array(float64, 1d, C))

During: resolving callee type: type(CPUDispatcher(<function gradient_boosting_objective_function at 0x7eff6c42ba60>))
During: typing of call at /mnt/c/Users/locke/Data/BE/realkd.py/realkd/search.py (732)


File "realkd/search.py", line 732:
def run_greedy_search_gradient_boosting(initial_extent, n, extents, g, h, reg):
    <source elided>
    extent = initial_extent
    value = gradient_boosting_objective_function(n, g, h, reg, extent)
    ^

  @jit
/mnt/c/Users/locke/Data/BE/realkd.py/realkd-env/lib/python3.8/site-packages/numba/core/object_mode_passes.py:151: 

test [ 1  4  5  7  9 10 11 13 15 18 19 20 22 23 26 28 29 33 34 36 39 42 43 46
 48 50 51 56 59 62 64 66 68 69 70 71 72 74 75 77 82 84 85 86 87 90 91 93
 94 96 97 98 99]
[]
[ 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5 -0.5 -0.5
 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5
 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5
 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5
 -0.5 -0.5]
a [] [ 1  4  5  7  9 10 11 13 15 18 19 20 22 23 26 28 29 33 34 36 39 42 43 46
 48 50 51 56 59 62 64 66 68 69 70 71 72 74 75 77 82 84 85 86 87 90 91 93
 94 96 97 98 99]
[]
[]
[ 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.

ValueError: Arguments must have the same data type.

In [ ]:
plt.subplots(1, 2, figsize=(10, 5))

t_numba_mean = [t_numba[(m, ns[0])].average for m in ms_to_plot]
t_numba_std = [t_numba[(m, ns[0])].stdev for m in ms_to_plot]
t_base_mean = [t_base[(m, ns[0])].average for m in ms_to_plot]
t_base_std = [t_base[(m, ns[0])].stdev for m in ms_to_plot]

plt.subplot(1, 2, 1)
plt.plot(ms_to_plot, t_numba_mean, marker='o')
plt.plot(ms_to_plot, t_base_mean, marker='*')
plt.xlim(ms_to_plot[0], ms_to_plot[-1])
plt.xlabel('$m$')

t_numba_mean = [t_numba[(ms[0], n)].average for n in ns_to_plot]
t_numba_std = [t_numba[(ms[0], n)].stdev for n in ns_to_plot]
t_base_mean = [t_base[(ms[0], n)].average for n in ns_to_plot]
t_base_std = [t_base[(ms[0], n)].stdev for n in ns_to_plot]

plt.subplot(1, 2, 2)
plt.plot(ns_to_plot, t_numba_mean, marker='o', label='Numbafied code')
plt.plot(ns_to_plot, t_base_mean, marker='*', label='Base case')
plt.xlim(ns_to_plot[0], ns_to_plot[-1])
plt.xlabel('$n$')
plt.legend()

plt.show()